In [ ]:
!pip install pandarallel
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
from pandarallel import pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=90ec48b506a7d514e7d63a74aa24bf206c5e195a22d4dd82da75bb760a4daab2
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.DataFrame()
read_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/diariodeibiza/url.csv', header=[0])
df=pd.concat([df,read_file]).reset_index(drop=True)
print(len(df))

15718


In [ ]:
def get_articles(url):
  try:
    r1 = requests.get(url)
    coverpage = r1.content
    soup = BeautifulSoup(coverpage, 'html.parser')
    
    try:
      coverpage=soup.find_all('div',class_='headline-article')
      title=coverpage[0].find('h1').get_text()
      try:
       description=coverpage[0].find('h2').get_text()
      except:
        description=''
    except:
      title=''
      description=''
    
    try:
      coverpage=soup.find_all('div',class_='article-entry-author')
      time=coverpage[0].find('span',class_='article-author__date').get_text()
    except:
      time=''  

    try:
      coverpage=soup.find_all('div',class_='article-body')
      article=''
      for i in coverpage[0].find_all('p'):
        article=article+i.get_text()
      # print(article)
    except:
      article=''
      print('article error\n'+url)


  except:
    title=''
    description=''
    time=''
    article=''
    print('url error\n'+url)    

  return [title,description,time,article]

In [ ]:
get_articles('https://www.diariodeibiza.es/internacional/2022/08/07/senado-eeuu-da-luz-verde-73157661.html')

['Victoria de Biden: el Senado aprueba su ley contra el cambio climático',
 'La normativa aumenta los impuestos a las grandes corporaciones para financiar las inversiones ambientales e incluye medidas para rebajar precios de medicamentos | El partido Demócrata aprueba la Ley de Reducción de la Inflación que prevé gravar con impuestos del 15% a grandes corporaciones para financiar su la inversión más grande en energía verde en la historia de EEUU',
 '\n                    07·08·22\n            ',
 '\n        El presidente Joe Biden y su partido han logrado este domingo que el Senado de EEUU apruebe el\xa0proyecto más ambicioso para incentivar la energía verde y la reducción de carbono. El proyecto de ley, llamado Ley de Reducción de la Inflación, representa la inversión en la lucha contra el cambio climático más grande en la historia de los EEUU, una victoria para Biden. La ley también supone cambios sustanciales en la política sanitaria al otorgar por primera vez a Medicare la negociac

In [ ]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

output=df['rows'].parallel_apply(get_articles)

df['country']='Diario de Ibiza'
df['title']=''
df['description']=''
df['time']=''
df['article']=''

for i in range(0,len(df)):
  df.loc[i]['title','description','time','article']=output[i]

df = df[df.title.str.len()!=0].reset_index(drop=True)
df = df.dropna()
str_name='/content/drive/MyDrive/Colab Notebooks/diariodeibiza/url_article.csv'
print(df)
df.to_csv(str_name, index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


article error
https://www.diariodeibiza.es/nacional/2022/07/19/equipo-inicial-nucleo-masculino-queda-68482371.html
article error
https://www.diariodeibiza.es/nacional/2021/11/16/informe-desvela-nuevo-herido-ojo-59584594.html
article error
https://www.diariodeibiza.es/nacional/2021/11/16/aragones-avisa-junts-negociacion-presupuestos-59584271.html
article error
https://www.diariodeibiza.es/nacional/2021/07/14/juez-ve-malversacion-sistematica-firma-55066857.html
article error
https://www.diariodeibiza.es/internacional/2021/05/02/protestas-proyecto-reforma-tributaria-colombia-50396598.html
article error
https://www.diariodeibiza.es/nacional/2022/05/19/llegada-rey-emerito-vigo-66300240.html
article error
https://www.diariodeibiza.es/nacional/2021/11/25/25n-mujeres-desaparecidas-violencia-machista-59937975.html
article error
https://www.diariodeibiza.es/nacional/2021/09/29/sido-llegada-lava-volcan-palma-57808445.html
article error
https://www.diariodeibiza.es/nacional/2021/07/12/nuevos-minis